In [6]:
import numpy as np
from parameter_estimator import ParameterEstimator
import utils
from scipy import linalg
from Pose_Estimation_Class import UKF

In [7]:
theta_nom = ParameterEstimator.dhparams["theta_nom"]
r_nom = ParameterEstimator.dhparams["r_nom"]
d_nom = ParameterEstimator.dhparams["d_nom"]
alpha_nom = ParameterEstimator.dhparams["alpha_nom"]

In [8]:
# pip4 = (np.pi / 4)*0.86
# pip2 = (np.pi / 2) * 0.65

In [9]:
# theta_nom1 = np.array([0,0,0,0,0,0,0]) + theta_nom
# T1 = ParameterEstimator.get_T__i0(7, theta_nom1, d_nom, r_nom, alpha_nom)

# theta_nom2 = np.array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]) + theta_nom
# T2 = ParameterEstimator.get_T__i0(7, theta_nom2, d_nom, r_nom, alpha_nom)

# theta_nom3 = np.array([0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]) + theta_nom
# T3 = ParameterEstimator.get_T__i0(7, theta_nom3, d_nom, r_nom, alpha_nom)

# theta_nom4 = np.array([0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3]) + theta_nom
# T4 = ParameterEstimator.get_T__i0(7, theta_nom4, d_nom, r_nom, alpha_nom)

# theta_nom5 = np.array([0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4]) + theta_nom
# T5 = ParameterEstimator.get_T__i0(7, theta_nom3, d_nom, r_nom, alpha_nom)

# theta_nom6 = np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]) + theta_nom
# T6 = ParameterEstimator.get_T__i0(7, theta_nom4, d_nom, r_nom, alpha_nom)

In [11]:
ukf = UKF()

In [25]:
X = np.array([[ 0.8660254,  0.0000000,  0.5000000, 0.0000000],
              [ 0.0000000,  1.0000000,  0.0000000, 0.0000000],
              [-0.5000000,  0.0000000,  0.8660254, 0.3000000],
              [ 0.0000000,  0.0000000,  0.0000000, 1.0000000]])
Xinv = np.linalg.inv(X)

In [23]:
utils.roundprint(X)

 0.866  0.000  0.500  0.000 

 0.000  1.000  0.000  0.000 

-0.500  0.000  0.866  0.300 

 0.000  0.000  0.000  1.000 





In [24]:
U = np.array([[-1.0000000,  0.0000000,  0.0000000, 2.0000000],
              [ 0.0000000, -1.0000000,  0.0000000, 2.0000000],
              [ 0.0000000,  0.0000000,  1.0000000, 1.0000000],
              [ 0.0000000,  0.0000000,  0.0000000, 1.0000000]])

In [44]:
for i in range(5):
    q_norm = np.random.rand(7)
    q = (q_norm - np.full((7,),0.5))*np.pi/2
    T1 = ParameterEstimator.get_T__i0(7, q, d_nom, r_nom, alpha_nom)
    M1 = Xinv@np.linalg.inv(T1)@U
    
    q_norm = np.random.rand(7)
    q = (q_norm - np.full((7,),0.5))*np.pi/2
    T2 = ParameterEstimator.get_T__i0(7, q, d_nom, r_nom, alpha_nom)
    M2 = Xinv@np.linalg.inv(T2)@U
    
    AA = M1@np.linalg.inv(M2)
    BB = np.linalg.inv(T1)@T2
    
    # test=AA@Xinv@np.linalg.inv(BB)@X
    # test = X@AA@Xinv@np.linalg.inv(BB)
    # utils.roundprint(test)
    
    ukf.Update(AA, BB)

0
 1.000  0.000  0.000  0.000 

 0.000  1.000  0.000  0.000 

 0.000  0.000  1.000  0.000 

 0.000  0.000  0.000  1.000 





LinAlgError: Array must not contain infs or NaNs